In [1]:
from disqco.circuits.cp_fraction import cp_fraction, cz_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.graphs.hypergraph_methods import calculate_full_cost_hetero
from qiskit import transpile
from disqco.parti.FM.FM_methods import set_initial_partitions
from disqco.circuits.QAOA import QAOA_random
from qiskit.circuit.library import QFT, QuantumVolume
import pickle
from copy import deepcopy
from disqco.graphs.quantum_network import *
from disqco.parti.FM.multilevel_FM import MLFM_recursive_hetero
import networkx as nx
from copy import deepcopy
import numpy as np
from disqco.graphs.quantum_network import QuantumNetwork
import os
import json

num_qubits_list = [128,256,512]
network_types = ['linear', 'grid']
num_partitions_and_coarsening_factors = [(4, 2), (8, 2), (16, 2), (32, 2), (64, 2), (8, 4), (16, 4), (32, 4), (64, 4), (16, 8), (32, 8), (64, 8)]

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_COARENING_POST_PROCESSED_nolimit.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

In [2]:
from disqco.parti.FM.net_coarsened_partitioning import run_full_net_coarsened_FM
from disqco.graphs.quantum_network import linear_coupling, grid_coupling
import time

iterations = 5
for iteration in range(iterations):
    for num_qubits in num_qubits_list:
        for network_type in network_types:
            for num_partitions_and_coarsening_factor in num_partitions_and_coarsening_factors:
                level_limit = 100  # Set the level limit for coarsening
                num_partitions, coarsening_factor = num_partitions_and_coarsening_factor
                print(f"num_qubits: {num_qubits}, network_type: {network_type}, num_partitions: {num_partitions}, coarsening_factor: {coarsening_factor}")
                # Check first whether the json file already contains results for this configuration
                existing_results = [entry for entry in detailed_results if entry['num_qubits'] == num_qubits and 
                                    entry['network_type'] == network_type and
                                    entry['num_partitions'] == num_partitions and
                                    entry['coarsening_factor'] == coarsening_factor]
                if existing_results:
                    if existing_results[0]['final_cost'] is not None:
                        print(f"Results already exist for num_qubits: {num_qubits}, network_type: {network_type}, num_partitions: {num_partitions}, coarsening_factor: {coarsening_factor}. Skipping this configuration.")
                        continue
                
                seed = np.random.randint(0, 10000)
                
                circuit = cp_fraction(num_qubits=num_qubits,
                                        depth=num_qubits,
                                        fraction=0.5,
                                        seed=seed)
                
                circuit = transpile(circuit, basis_gates=['cp', 'u'])

                qpu_sizes = [int(np.ceil(num_qubits / num_partitions)) + 1 for i in range(num_partitions)]

                if network_type == 'linear':
                    coupling = linear_coupling(num_partitions)
                elif network_type == 'grid':
                    coupling = grid_coupling(num_partitions)

                initial_qpu_sizes = {i: qpu_sizes[i] for i in range(num_partitions)}
                initial_network = QuantumNetwork(qpu_sizes, coupling)

                network = deepcopy(initial_network)
                initial_graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True)
                depth = initial_graph.depth
                graph_no_grouping = QuantumCircuitHyperGraph(circuit, group_gates=False, anti_diag=True)

                graph = deepcopy(initial_graph)
                assignment = set_initial_partitions(network, num_qubits, depth)

                initial_cost_no_grouping = calculate_full_cost_hetero(hypergraph=graph_no_grouping, assignment=assignment, num_partitions=num_partitions, costs = {}, network=network)

                print("Initial cost with no grouping: ", initial_cost_no_grouping)

                initial_cost = calculate_full_cost_hetero(hypergraph=graph, assignment=assignment, num_partitions=num_partitions, costs = {}, network=network)

                print("Initial Cost: ", initial_cost)

                try:
                    start_time = time.time()
                    results  = run_full_net_coarsened_FM(
                        circuit=circuit,
                        num_qubits=num_qubits,
                        network=network,
                        coarsening_factor=coarsening_factor,
                        passes_per_level=10,
                        use_multiprocessing=True,
                        ML_internal_level_limit=100)
                    
                    final_assignment = results['final_assignment']
                    final_cost = results['final_cost']

                    end_time = time.time()
                    print("Final Cost: ", final_cost)
                    print("Time taken: ", end_time - start_time)
                    results_entry = {
                        "num_qubits": num_qubits,
                        "network_type": network_type,
                        "num_partitions": num_partitions,
                        "coarsening_factor": coarsening_factor,
                        "level_limit": level_limit,
                        "initial_cost_no_grouping": initial_cost_no_grouping,
                        "initial_cost": initial_cost,
                        "final_cost": final_cost,
                        "iteration": iteration,
                        "time_taken": end_time - start_time,
                        "seed": seed
                    }
                    detailed_results.append(results_entry)
                except Exception as e:
                    raise e
                    print(f"Error occurred: {e}")
                    results_entry = {
                        "num_qubits": num_qubits,
                        "network_type": network_type,
                        "num_partitions": num_partitions,
                        "coarsening_factor": coarsening_factor,
                        "level_limit": level_limit,
                        "initial_cost_no_grouping": initial_cost_no_grouping,
                        "initial_cost": initial_cost,
                        "final_cost": None,
                        "iteration": iteration,
                        "time_taken": None,
                        "seed": seed
                    }
                    detailed_results.append(results_entry)
                    if level_limit > 1:
                        level_limit -= 1
                        
                        
                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)


num_qubits: 128, network_type: linear, num_partitions: 4, coarsening_factor: 2
Number of layers: 130
Number of layers: 130
Initial cost with no grouping:  5002
Initial Cost:  3270
Number of layers: 130
Processing level 0/1
Number of layers: 130
Best cost at level 0: 1095
Best cost at level 1: 1048
Best cost at level 2: 986
Best cost at level 3: 923
Best cost at level 4: 887
Best cost at level 5: 850
Best cost at level 6: 834
Best cost at level 7: 827
Processing level 1/1
Final Cost:  2165
Time taken:  12.213182926177979
num_qubits: 128, network_type: linear, num_partitions: 8, coarsening_factor: 2
Number of layers: 130
Number of layers: 130
Initial cost with no grouping:  10212
Initial Cost:  6493
Number of layers: 130
Processing level 0/2
Number of layers: 130
Best cost at level 0: 1059
Best cost at level 1: 1037
Best cost at level 2: 970
Best cost at level 3: 919
Best cost at level 4: 883
Best cost at level 5: 851
Best cost at level 6: 836
Best cost at level 7: 831
Processing level 1

In [2]:
# Now set up same testing loop using standard MLFM-R
# Read through JSON file to find random seeds to generate each circuit
import time


from disqco.parti.FM.multilevel_FM import MLFM_recursive_hetero

# final_assignment_list, final_cost_list, _ = MLFM_recursive_hetero(graph,
#                                                                     assignment,
#                                                                     qpu_sizes,
#                                                                     limit=num_qubits,
#                                                                     network=network,
#                                                                     log=True,
#                                                                     stochastic=True,
#                                                                     costs=costs, 
#                                                                     level_limit=None)


with open(detailed_filename, "r") as f:
    detailed_results = json.load(f)


direct_partitioning_filename = "benchmark_results_COARSENING_POST_PROCESSED_direct_partitioning.json"
if os.path.exists(direct_partitioning_filename):
    with open(direct_partitioning_filename, "r") as f:
        direct_results = json.load(f)
else:
    direct_results = []

completed_set = set()

for result in direct_results:
    num_qubits = result['num_qubits']
    network_type = result['network_type']
    num_partitions = result['num_partitions']
    completed_set.add((num_qubits, network_type, num_partitions))

for result in detailed_results:

    num_qubits = result['num_qubits']
    network_type = result['network_type']
    num_partitions = result['num_partitions']
    level_limit = result['level_limit']
    initial_cost_no_grouping = result['initial_cost_no_grouping']
    initial_cost = result['initial_cost']
    final_cost = result['final_cost']
    iteration = result['iteration']
    time_taken = result['time_taken']
    seed = result['seed']
    if num_qubits is not 512 and network_type != 'grid' and num_partitions != 4:
        continue
    if (num_qubits, network_type, num_partitions) in completed_set:
        print(f"Skipping already completed configuration: num_qubits={num_qubits}, network_type={network_type}, num_partitions={num_partitions}")
        continue
    completed_set.add((num_qubits, network_type, num_partitions))
    
    if num_partitions <= 16:
        level_limit = 8
    else:
        results_entry = {
            "num_qubits": num_qubits,
            "network_type": network_type,
            "num_partitions": num_partitions,
            "level_limit": 1,
            "initial_cost_no_grouping": initial_cost_no_grouping,
            "initial_cost": initial_cost,
            "final_cost": initial_cost,  # No final cost for direct partitioning as too large
            "iteration": iteration,
            "time_taken": None,
            "seed": seed
        }

        direct_results.append(results_entry)
        with open(direct_partitioning_filename, "w") as f:
            json.dump(direct_results, f, indent=2)
        print(f"Skipping direct partitioning for num_qubits: {num_qubits}, network_type: {network_type}, num_partitions: {num_partitions} as level_limit is 1")
        continue
    
    print(f"Direct partitioning for num_qubits: {num_qubits}, network_type: {network_type}, num_partitions: {num_partitions}, level_limit: {level_limit}")
    circuit = cp_fraction(num_qubits=num_qubits,
                            depth=num_qubits,
                            fraction=0.5,
                            seed=seed)
    
    circuit = transpile(circuit, basis_gates=['cp', 'u'])

    qpu_sizes = [int(np.ceil(num_qubits / num_partitions)) + 1 for i in range(num_partitions)]

    if network_type == 'linear':
        coupling = linear_coupling(num_partitions)
    elif network_type == 'grid':
        coupling = grid_coupling(num_partitions)

    initial_qpu_sizes = {i: qpu_sizes[i] for i in range(num_partitions)}
    network = QuantumNetwork(qpu_sizes, coupling)


    graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True)

    depth = graph.depth

    assignment = set_initial_partitions(network, num_qubits, depth)
    start_time = time.time()
    final_assignment_list, final_cost_list, _ = MLFM_recursive_hetero(graph,
                                                                    assignment,
                                                                    qpu_sizes,
                                                                    limit=num_qubits,
                                                                    network=network,
                                                                    log=True,
                                                                    stochastic=True,
                                                                    costs={},
                                                                    level_limit=level_limit)
    end_time = time.time()

    time_taken = end_time - start_time
    
    results_entry = {
        "num_qubits": num_qubits,
        "network_type": network_type,
        "num_partitions": num_partitions,
        "level_limit": level_limit,
        "initial_cost_no_grouping": initial_cost_no_grouping,
        "initial_cost": initial_cost,
        "final_cost": min(final_cost_list),
        "iteration": iteration,
        "time_taken": time_taken,
        "seed": seed}
    
    direct_results.append(results_entry)

    with open(direct_partitioning_filename, "w") as f:
        json.dump(direct_results, f, indent=2)


<>:50: SyntaxWarning: "is not" with 'int' literal. Did you mean "!="?
<>:50: SyntaxWarning: "is not" with 'int' literal. Did you mean "!="?
/var/folders/s5/7cd6cqh90fd7qlk1kl1bnr0w0000gp/T/ipykernel_56717/3993395770.py:50: SyntaxWarning: "is not" with 'int' literal. Did you mean "!="?
  if num_qubits is not 512 and network_type != 'grid' and num_partitions != 4:


Skipping already completed configuration: num_qubits=128, network_type=linear, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=8
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=16
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=32
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=64
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=8
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=16
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=32
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=64
Skipping already completed configuration: num_qubits

TypeError: calculate_full_cost_hetero() got an unexpected keyword argument 'assignment_map'. Did you mean 'assignment'?